In [ ]:
import numpy as np
import time
import matplotlib.pyplot as plt

# Define spin operators
Sz = np.array([[0.5,  0], [0, -0.5]])
Sp = np.array([[0, 1], [0, 0]])
Sm = np.array([[0, 0], [1, 0]])
I = np.eye(2)

# Parameters
Nit = 50  # Number of iterations
D_values = [8, 12]  # Different bond dimensions
tol_deg = 1e-10  # Tolerance value for eigenvalue cutoff
threshold = 1e-3

plt.figure(figsize=(8, 6))

for D in D_values:
    site_list = []
    energy_gap_list = []
    H_expectation_list = []  # List for Hamiltonian expectation values
    Sz_expectation_list = []  # List for Total Magnetization values
    correlation_list = []  # List for correlation function values
    start_time = time.time()
    
    for k in range(1, Nit + 1):
        if k == 1:
            I_S, I_E = I, I
            H_S, H_E = I_S * 0, I_E * 0
            Szborder_S, Spborder_S, Smborder_S = Sz, Sp, Sm
            Szborder_E, Spborder_E, Smborder_E = Sz, Sp, Sm
        else:
            dim_L, dim_R = np.shape(I_L)[0], np.shape(I_R)[0]
            psi_mat = np.reshape(psi, (dim_L, dim_R))
            
            # Left block truncation
            sigma_L = np.matmul(psi_mat, psi_mat.conj().T)
            vals_L, vecs_L = np.linalg.eigh(sigma_L)
            O_L = vecs_L[:, -1].reshape(-1, 1)
            for it in range(1, min(D, len(vals_L))):
                O_L = np.hstack([O_L, vecs_L[:, -1-it].reshape(-1, 1)])
            
            # Right block truncation
            sigma_R = np.matmul(psi_mat.T, psi_mat.conj())
            vals_R, vecs_R = np.linalg.eigh(sigma_R)
            O_R = vecs_R[:, -1].reshape(-1, 1)
            for it in range(1, min(D, len(vals_R))):
                O_R = np.hstack([O_R, vecs_R[:, -1-it].reshape(-1, 1)])
            
            # Transform operators
            I_S, H_S = O_L.T @ I_L @ O_L, O_L.T @ H_L @ O_L
            Szborder_S = O_L.T @ Szborder_L @ O_L
            I_E, H_E = O_R.T @ I_R @ O_R, O_R.T @ H_R @ O_R
            Szborder_E = O_R.T @ Szborder_R @ O_R
        
        # Extend the blocks
        I_L, H_L = np.kron(I_S, I), np.kron(H_S, I) + np.kron(Szborder_S, Sz)
        Szborder_L = np.kron(I_S, Sz)
        I_R, H_R = np.kron(I, I_E), np.kron(I, H_E) + np.kron(Sz, Szborder_E)
        Szborder_R = np.kron(Sz, I_E)
        
        # Construct the full Hamiltonian
        H_SB = np.kron(H_L, I_R) + np.kron(I_L, H_R) + np.kron(Szborder_L, Szborder_R)
        
        # Diagonalization
        En, psin = np.linalg.eigh(H_SB)
        E0 = En[0]  # Ground state energy
        E1 = En[1]  # First excited state energy
        energy_gap = E1 - E0  # Energy gap
        psi = psin[:, 0]
        
        # Compute expectation value of the Hamiltonian
        H_expectation = np.real(np.vdot(psi, H_SB @ psi))
        
        # Compute expectation value of total magnetization (Sz_total)
        Sz_total = np.kron(Szborder_L, I_R) + np.kron(I_L, Szborder_R)
        Sz_expectation = np.real(np.vdot(psi, Sz_total @ psi))
        
        # Compute correlation function ⟨Sz_i Sz_j⟩ - ⟨Sz_i⟩⟨Sz_j⟩
        Sz_i = np.kron(Szborder_L, I_R)
        Sz_j = np.kron(I_L, Szborder_R)
        Sz_i_expectation = np.real(np.vdot(psi, Sz_i @ psi))
        Sz_j_expectation = np.real(np.vdot(psi, Sz_j @ psi))
        Sz_correlation = np.real(np.vdot(psi, (Sz_i @ Sz_j) @ psi)) - Sz_i_expectation * Sz_j_expectation
        
        # Store results
        Nsites = 2 + 2 * k
        site_list.append(Nsites)
        energy_gap_list.append(energy_gap)
        H_expectation_list.append(H_expectation)
        Sz_expectation_list.append(Sz_expectation)
        correlation_list.append(Sz_correlation)
        
        print(f"D = {D}, #sites = {Nsites}, ⟨H⟩ = {H_expectation:.6f}, ⟨Sz⟩ = {Sz_expectation:.6f}, ⟨Sz_i Sz_j⟩ - ⟨Sz_i⟩⟨Sz_j⟩ = {Sz_correlation:.6f}")
        start_time = time.time()
    
    plt.plot(site_list, energy_gap_list, marker='o', label=f'D={D} Energy Gap')
    
# Plot results
plt.xlabel("Number of sites")
plt.ylabel("Energy Gap (ΔE)")
plt.title("Energy Gap Between Ground and First Excited State")
plt.legend()
plt.grid()
plt.show()